In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Workspace

ws = Workspace.from_config()
ws.write_config(path='.azureml')

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-147095
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-147095


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

comp_cluster = 'opt-ml-cluster'
try:
    comp_trget = ComputeTarget(workspace=ws,name=comp_cluster)
except ComputeTargetException:  
    comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_v2', max_nodes=4, min_nodes=1)
comp_trget = ComputeTarget.create(ws, comp_cluster, comp_confg)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform , choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})


# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =  SKLearn(
    source_directory = '.',
    compute_target = comp_trget,
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    estimator=est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a
Web View: https://ml.azure.com/runs/HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-147095/workspaces/quick-starts-ws-147095&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
AmlCompute is getting created. Consider calling wait_for_completion() first


Streaming azureml-logs/hyperdrive.txt

"<START>[2021-06-13T16:06:45.219272][API][INFO]Experiment created<END>\n""<START>[2021-06-13T16:06:45.694331][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-06-13T16:06:45.880326][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a
Web View: https://ml.azure.com/runs/HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-147095/workspaces/quick-st

{'runId': 'HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a',
 'target': 'opt-ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-06-13T16:06:44.968408Z',
 'endTimeUtc': '2021-06-13T16:16:23.051294Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'prim

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ', best_run.id )
print('Metrics : ', best_run.get_metrics())
#joblib.dump(best_run,'model.joblib')
#model = best_run.register_model(model_name="best_model",model_path="./outputs/best_model.joblib")

ID :  HD_7a3e0769-8ee6-4228-bcd7-80f44b8f0e9a_0
Metrics :  {'Regularization Strength:': 0.03, 'Max iterations:': 50, 'Accuracy': 0.9118866970156804}


In [6]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_67ec9b632dd5d9ee5ee69a3e22c01354fc5c280d11aa2202dbe0acf9360ae38e_d.txt',
 'azureml-logs/65_job_prep-tvmps_67ec9b632dd5d9ee5ee69a3e22c01354fc5c280d11aa2202dbe0acf9360ae38e_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_67ec9b632dd5d9ee5ee69a3e22c01354fc5c280d11aa2202dbe0acf9360ae38e_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/107_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [7]:
best_run.download_file(best_run.get_file_names()[-1],output_file_path='./outputs/')

In [8]:
joblib.load('./outputs/model.joblib')

The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.


LogisticRegression(C=0.03, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dir_web=  "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
input_data = TabularDatasetFactory.from_delimited_files(path=dir_web)

In [11]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
pred_col = pd.DataFrame(y)
pred_col.head()
df = pd.concat([x ,pred_col  ],axis=1)
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [12]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,test_size=0.3)
train_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
23489,43,0,0,1,0,5,5,618,2,999,...,1,1,0,0,0,0,0,0,0,1
19951,34,1,0,0,0,11,3,363,2,999,...,0,0,0,0,0,0,0,1,0,0
22843,27,0,0,1,0,10,3,184,1,999,...,0,0,0,0,0,0,0,1,0,1
18604,56,1,0,1,0,8,2,370,2,999,...,0,0,0,0,0,0,0,1,0,1
11744,32,1,0,0,0,5,2,102,4,999,...,1,0,0,1,0,0,0,0,0,0


In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name='y',
    n_cross_validations=7)

In [14]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl= exp.submit(automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_2ff910b1-ba28-4aea-b192-29db86f237ed,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_class, best_fit = class_automl.get_output()
print(best_fit)
joblib.dump(best_fit,'./outputs/bestmodel.joblib')
best_class.register_model(model_name='bestmodel.pkl',model_path='/outputs/')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.7291666666666667, subsample=0.9, tree_method='auto'))], verbose=False)), ('9', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=2.559547922699533, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('randomforestclassifier', Rando

Model(workspace=Workspace.create(name='quick-starts-ws-147095', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-147095'), name=bestmodel.pkl, id=bestmodel.pkl:2, version=2, tags={}, properties={})